# Python to Excel

## Importing our libraries

In [1]:
import pandas as pd
import xlsxwriter

In [2]:
data=pd.read_csv("C:/Users/user/ABNBstocks.csv", parse_dates=True, index_col= 0)
data.head()

,High,Low,Open,Close,% -chg,Daily_returns,Log_returns,SMA40,SMA200,EMA40,MACD,Signal line,%k,%D
Date,,,,,,,,,,,,,,
2020-12-10,165.000000,141.250000,146.000000,144.710007,NaN,-0.037731,NaN,NaN,NaN,144.710007,0.000000,0.000000,NaN,NaN
2020-12-11,151.500000,135.100006,146.550003,139.250000,-0.037731,-0.066427,-0.038461,NaN,NaN,144.443665,-0.435556,-0.087111,NaN,NaN
2020-12-14,135.300003,125.160004,135.000000,130.000000,-0.066427,-0.040000,-0.068736,NaN,NaN,143.739096,-1.509732,-0.371635,NaN,NaN
2020-12-15,127.599998,121.500000,126.690002,124.800003,-0.040000,0.105689,-0.040822,NaN,NaN,142.815238,-2.748932,-0.847095,NaN,NaN
2020-12-16,142.000000,124.910004,125.830002,137.990005,0.105689,0.065657,0.100469,NaN,NaN,142.579861,-2.636295,-1.204935,NaN,NaN


In [3]:
#lets extract data from 2022 to current period 

df1=data.loc["2022-01-01":]
df1=df1.iloc[::-1] #reverse the order so we have the most recent date first
df1.head()


,High,Low,Open,Close,% -chg,Daily_returns,Log_returns,SMA40,SMA200,EMA40,MACD,Signal line,%k,%D
Date,,,,,,,,,,,,,,
2022-05-24,111.419998,105.349998,111.279999,105.900002,-0.065148,NaN,-0.067367,113.941000,150.388401,138.258420,-12.662574,-11.641932,1.133326,7.481178
2022-05-23,115.150002,110.930000,113.544998,113.279999,0.006486,-0.065148,0.006465,114.964000,151.040401,139.917826,-12.440892,-11.386772,11.366740,11.470726
2022-05-20,116.250000,108.500000,115.459999,112.550003,-0.014189,0.006486,-0.014291,115.573000,151.677601,141.283868,-12.728406,-11.123241,9.943469,7.927525
2022-05-19,116.349998,107.449997,108.180000,114.169998,0.056836,-0.014189,0.055280,117.902000,152.462601,142.757400,-12.835910,-10.721950,13.101969,8.637128
2022-05-18,117.040001,107.620003,116.110001,108.029999,-0.080596,0.056836,-0.084029,120.793999,153.145401,144.223421,-12.951366,-10.193460,0.737138,6.333114


## Using Pandas ExcelWriter to write to Excel 

In [4]:
writer= pd.ExcelWriter("Technical_analysis_ABNB.xlsx", engine= "xlsxwriter", 
                       date_format="yyyy-mm-dd", datetime_format="yyyy-mm-dd")

workbook= writer.book

#creating format for red and green cells
red_fill= workbook.add_format({'bg_color': '#FFC7CE',
                                'font_color': '#9C0006'})


green_fill= workbook.add_format({'bg_color': '#C6EFCE',
                                'font_color':'#006100'})


#first sheet in our wookbook
sheet_name= "Close v. SMA40 v. SMA200"
sma_df= df1.loc[:,["Close","SMA40", "SMA200"]]
sma_df.to_excel(writer, sheet_name=sheet_name)
worksheet= writer.sheets[sheet_name]

#adjusting the width of A column
worksheet.set_column("A:A",13)

for col in range(1,3):   #reiterating over the columns...from the second column to the third column
                            #the colors will be applied to these columns...remember we use python's indexing which starts from 0
    worksheet.conditional_format(1,col,len(sma_df),col
                                 #first row, column num, last row, column num
                                 #if closing price is greater than sma40, it will be colored green
                                 ,{'type': 'formula',
                                'criteria': '=B2>= C2',
                                'format': green_fill})

    #format for red cell
    worksheet.conditional_format(1,col,len(sma_df),col,   #if closing price is less than sma40, it will be colored red
                                {'type': 'formula',
                                 'criteria': '=B2<C2',
                                 'format': red_fill})
    
worksheet.conditional_format(1,3,len(sma_df),3,       #if closing price is greater than sma200, sma200 will be colored green
                            {'type':'formula',
                            'criteria': '=B2>=D2',
                            'format':green_fill})

worksheet.conditional_format(1,3,len(sma_df),3,      #if closing price is less than sma200, sma200 will be colored red
                            {'type':'formula',
                            'criteria':'=B2<D2',
                            'format':red_fill})

chart1= workbook.add_chart({
    'type':'line'})

chart1.add_series({
 'name': "Close Price",
    'categories': [sheet_name, 1,0, len(sma_df),0],
    #categories is similar to x axis 
    #sheetname, first row, col_num, last row, col_num
    'values':[sheet_name, 1,1,len(sma_df),1]})
    
chart2=workbook.add_chart({
    'type':'line'})

chart2.add_series({
    'name': 'SMA40',
    'categories': [sheet_name, 1,0, len(sma_df),0],
    'values':[sheet_name,1,2,len(sma_df),2]
})

chart1.combine(chart2)  #note that you can only combine 2 charts when using xlsxwriter


chart1.set_title({'name': "Close v. SMA40 ABNB "})
chart1.set_x_axis({'name':'Date'})
chart1.set_y_axis({'name': 'Price'})
chart1.set_size({'width': 720, 'height': 380})

worksheet.insert_chart('G2',chart1)


#second sheet
sheet_name= "MACD v. Signal line"
macd_df= df1.loc[:, ["Close", "MACD", "Signal line"]]
macd_df.to_excel(writer, sheet_name=sheet_name)
worksheet=writer.sheets[sheet_name]

worksheet.set_column("A:A", 13)

for col in range(1,4):
    worksheet.conditional_format(1,col,len(macd_df),col,  #if macd greater than signal line, color it green
                                {'type':'formula',
                                'criteria':'=C2>=D2',
                                'format':green_fill})
    
    worksheet.conditional_format(1,col,len(macd_df),col, #if macd less than signal line, color it red
                                {'type':'formula',
                                'criteria': '=C2<D2',
                                'format':red_fill})
    

chart1= workbook.add_chart({
    'type':'line'})

chart1.add_series({
    'name': 'MACD',
    'categories':[sheet_name, 1,0,len(macd_df),0],
    'values':[sheet_name,1,2,len(macd_df),2]})

chart2=workbook.add_chart({
    'type':'line'})

chart2.add_series({
    'name':'Signal line',
    'categories':[sheet_name,1,0,len(macd_df),0],
    'values':[sheet_name,1,3,len(macd_df),3]})

chart1.combine(chart2)

chart1.set_title({'name':'MACD v. Signal line ABNB'})
chart1.set_x_axis({'name':'Date',
                  'label_position':'low',
                  'num_font':{'rotation':30}
                  })
chart1.set_y_axis({'name': 'value'})
chart1.set_size({'width': 720, 'height': 380})

worksheet.insert_chart('G2', chart1)


#third sheet
sheet_name= "Stochastic Oscillator"
sto_df= df1.loc[:, ["Close", "%k", "%D"]]
sto_df.to_excel(writer, sheet_name=sheet_name)
worksheet=writer.sheets[sheet_name]

worksheet.set_column("A:A", 13)

for col in range(1,4):
    worksheet.conditional_format(1,col,len(sto_df), col,  #green color if %k greater than %D
                                {'type':'formula',
                                'criteria': '=C2>=D2',
                                'format':green_fill})
    
    worksheet.conditional_format(1,col,len(sto_df), col, #red color if %k less than %D
                                {'type':'formula',
                                'criteria': '=C2<D2',
                                'format':red_fill})
    
chart1=workbook.add_chart({
    'type':'line'})

chart1.add_series({
    'name': '%k',
    'categories':[sheet_name, 1,0, len(sto_df),0],
    'values':[sheet_name,1,2,len(sto_df),2]})

chart2= workbook.add_chart({
    'type':'line'})

chart2.add_series({
    'name': '%D',
    'categories': [sheet_name,1,0,len(sto_df),0],
    'values': [sheet_name,1,3,len(sto_df),3]})

chart1.combine(chart2)

chart1.set_title({'name': "%k v. %D ABNB"})
chart1.set_x_axis({'name': 'Date',
                  'label_position':'low',
                  'num_font':{'rotation': 30}})
chart1.set_y_axis({'name': 'values'})
chart1.set_size({'width': 720, 'height': 380})

worksheet.insert_chart('G2', chart1)
    

writer.close()    